### Manipulation of PeaskDB de novo-assisted database search results of _T. weisflogii_ rot samples LC-MS/MS data using python.

Starting with:

    PeaksDB search results (.csv) of PTM-optimized database searches
    These were all searched with 15 ppm precursor tolerance and 0.5 ppm fragement ion tolerance
    Multiple injections and fragmentation strategies included
    Exported at <1.0% FDR
    
Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use for a different file:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the running # (e.g., 233) for another
#### 3. Update the NAAF factor calculated in *IN 6* into *IN 7*

In [1]:
cd /home/millieginty/Documents/git-repos/rot-mayer/data/MED_Weissrot_Fusion_UWPR2021/MED_Weissrot_Fusion_322-T0dig-all_PEAKS_75/

/home/millieginty/Documents/git-repos/rot-mayer/data/MED_Weissrot_Fusion_UWPR2021/MED_Weissrot_Fusion_322-T0dig-all_PEAKS_75


In [2]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [3]:
ls 

322-T0dig-all_PEAKS_75_DB-search-psm.csv
322-T0dig-all_PEAKS_75_DNO.csv
322-T0dig-all_PEAKS_75_DNO.xml
322-T0dig-all_PEAKS_75_peptide.csv
322-T0dig-all_PEAKS_75_peptides.pep.xml
322-T0dig-all_PEAKS_75_peptide.xlsx
322-T0dig-all_PEAKS_75_protein-peptides.csv
322-T0dig-all_PEAKS_75_proteins.csv
322-T0dig-all_PEAKS_75_proteins.fasta
322-T0dig-all_PEAKS_75_proteins-noX.fasta
322-T0dig-all_PEAKS_75_proteins_trypsin_digested_Mass400to6000.txt
322-T0dig-all_PEAKS_75_proteins.xlsx


In [4]:
# read the CSV into a dataframe using the pandas read_csv function
peaksdbdup322 = pd.read_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/MED_Weissrot_Fusion_UWPR2021/MED_Weissrot_Fusion_322-T0dig-all_PEAKS_75/322-T0dig-all_PEAKS_75_peptide.csv")

# remove redundant rows
peaksdb322 = pd.DataFrame.drop_duplicates(peaksdbdup322)

print(peaksdb322.columns)

columns = ['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area', 'Fraction', 'Scan', 'Source_File',
       '#Spec', '#Spec', 'Accession', 'PTM',
       'AScore']

peaksdb322.columns = columns

#remmove # spec and accession columns because they mess parsing up

del peaksdb322['#Spec']
del peaksdb322['Accession']
del peaksdb322['PTM']
del peaksdb322['AScore']

mean_length = peaksdb322['Length'].mean()
print(mean_length)

print("# redundant peaksdb peptides in combined dataframe", len(peaksdbdup322))
print("# nonredundant peaksdb peptides in combined dataframe", len(peaksdb322))

#look at the dataframe
peaksdb322.head()

Index(['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area 322_T0_trypsin', 'Fraction', 'Scan', 'Source File', '#Spec',
       '#Spec 322_T0_trypsin', 'Accession', 'PTM', 'AScore'],
      dtype='object')
10.05084745762712
# redundant peaksdb peptides in combined dataframe 826
# nonredundant peaksdb peptides in combined dataframe 826


,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,Source_File
0,LPQVEGTGGDVQPSQDLVR,59.68,1994.0068,19,0.3,665.6764,75.45,134000000.0,4,15770,20210114_Weissrot_322_T0_trypsin_EThcD_120min_...
1,VIGQNEAVDAVSNAIR,57.24,1654.8638,16,0.8,552.6290,90.87,142000000.0,3,18509,20210113_Weissrot_322_T0_trypsin_EThcD_120min_...
2,AIDLIDEAASSIR,54.60,1372.7197,13,0.2,458.5806,108.20,171000000.0,3,22017,20210113_Weissrot_322_T0_trypsin_EThcD_120min_...
3,VTDAEIAEVLAR,50.81,1285.6877,12,0.1,429.5699,97.73,192000000.0,3,19961,20210113_Weissrot_322_T0_trypsin_EThcD_120min_...
4,STEFDNILIVGPIAGK,49.60,1672.9036,16,0.0,837.4590,110.83,78300.0,1,34719,20210112_Weissrot_322_T0_trypsin_DDA_120min_2u...


In [5]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaksdb322['A'] = peaksdb322['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaksdb322['C'] = peaksdb322['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaksdb322['D'] = peaksdb322['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaksdb322['E'] = peaksdb322['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaksdb322['F'] = peaksdb322['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaksdb322['G'] = peaksdb322['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaksdb322['H'] = peaksdb322['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaksdb322 output, there will be no isoleucines (they're lumped in with leucines)
peaksdb322['I'] = peaksdb322['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaksdb322['K'] = peaksdb322['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaksdb322['L'] = peaksdb322['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaksdb322['M'] = peaksdb322['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaksdb322['N'] = peaksdb322['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaksdb322['P'] = peaksdb322['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaksdb322['Q'] = peaksdb322['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaksdb322['R'] = peaksdb322['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaksdb322['S'] = peaksdb322['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaksdb322['T'] = peaksdb322['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaksdb322['V'] = peaksdb322['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaksdb322['W'] = peaksdb322['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaksdb322['Y'] = peaksdb322['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaksdb322['c-carb'] = peaksdb322['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaksdb322['m-oxid'] = peaksdb322['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a count function to enumerate the # of oxidized K's in each peptide
peaksdb322['k-oxid'] = peaksdb322['Peptide'].apply(lambda x: x.count('K(+15.99)'))

# use a count function to enumerate the # of oxidized P's in each peptide
peaksdb322['p-oxid'] = peaksdb322['Peptide'].apply(lambda x: x.count('P(+15.99)'))

# use a count function to enumerate the # of oxidized R's in each peptide
peaksdb322['r-oxid'] = peaksdb322['Peptide'].apply(lambda x: x.count('R(+15.99)'))

# use a count function to enumerate the # of oxidized Y's in each peptide
peaksdb322['y-oxid'] = peaksdb322['Peptide'].apply(lambda x: x.count('Y(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaksdb322['n-deam'] = peaksdb322['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a count function to enumerate the # of methylated K's in each peptide
peaksdb322['k-meth'] = peaksdb322['Peptide'].apply(lambda x: x.count('K(+14.02)'))

# use a count function to enumerate the # of methylated R's in each peptide
peaksdb322['r-meth'] = peaksdb322['Peptide'].apply(lambda x: x.count('R(+14.02)'))

# use a count function to enumerate the # of pyro glu Q's in each peptide
peaksdb322['q-pyro'] = peaksdb322['Peptide'].apply(lambda x: x.count('Q(-17.03)'))

# use a count function to enumerate the # of acetylation of K's in each peptide
peaksdb322['k-acet'] = peaksdb322['Peptide'].apply(lambda x: x.count('K(+42.01)'))

# create a column with 'stripped' peptide sequences using strip
peaksdb322['stripped_peptide'] = peaksdb322['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaksdb322['stripped_length'] = peaksdb322['stripped_peptide'].apply(len)

peaksdb322['NAAF_num.'] = peaksdb322['Area'] / peaksdb322['stripped_length']

# total the number of modifications in sequence
peaksdb322['ptm-total'] = peaksdb322['c-carb'] + peaksdb322['m-oxid'] + peaksdb322['k-oxid'] + peaksdb322['p-oxid'] \
+ peaksdb322['r-oxid'] + peaksdb322['y-oxid'] + peaksdb322['n-deam'] + peaksdb322['k-meth'] + peaksdb322['r-meth'] \
+ peaksdb322['q-pyro'] + peaksdb322['k-acet']

# turn all isoleucines into leucines
# this helps later in comparing Unipept peptides to PeaksDB and Comet ones
peaksdb322['stripped_IL']= peaksdb322['stripped_peptide'].str.replace('I','L')

# write modified dataframe to new txt file
peaksdb322.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDB/TW_322_T0_trypsin_combine_PTMopt_DB_FDR1.csv")

# check out the results
peaksdb322.head()

,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,...,n-deam,k-meth,r-meth,q-pyro,k-acet,stripped_peptide,stripped_length,NAAF_num.,ptm-total,stripped_IL
0,LPQVEGTGGDVQPSQDLVR,59.68,1994.0068,19,0.3,665.6764,75.45,134000000.0,4,15770,...,0,0,0,0,0,LPQVEGTGGDVQPSQDLVR,19,7.052632e+06,0,LPQVEGTGGDVQPSQDLVR
1,VIGQNEAVDAVSNAIR,57.24,1654.8638,16,0.8,552.6290,90.87,142000000.0,3,18509,...,0,0,0,0,0,VIGQNEAVDAVSNAIR,16,8.875000e+06,0,VLGQNEAVDAVSNALR
2,AIDLIDEAASSIR,54.60,1372.7197,13,0.2,458.5806,108.20,171000000.0,3,22017,...,0,0,0,0,0,AIDLIDEAASSIR,13,1.315385e+07,0,ALDLLDEAASSLR
3,VTDAEIAEVLAR,50.81,1285.6877,12,0.1,429.5699,97.73,192000000.0,3,19961,...,0,0,0,0,0,VTDAEIAEVLAR,12,1.600000e+07,0,VTDAELAEVLAR
4,STEFDNILIVGPIAGK,49.60,1672.9036,16,0.0,837.4590,110.83,78300.0,1,34719,...,0,0,0,0,0,STEFDNILIVGPIAGK,16,4.893750e+03,0,STEFDNLLLVGPLAGK


In [6]:
# keep only stripped peptide I/L and NAAF
db_322 = peaksdb322[['stripped_IL', 'Area', 'NAAF_num.']]

db_322.set_index('stripped_IL')

# write modified dataframe to new txt file
db_322.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/I-L_NAAFs/T0_322_trypsin_DB_peptides.csv")

db_322.head()

,stripped_IL,Area,NAAF_num.
0,LPQVEGTGGDVQPSQDLVR,134000000.0,7.052632e+06
1,VLGQNEAVDAVSNALR,142000000.0,8.875000e+06
2,ALDLLDEAASSLR,171000000.0,1.315385e+07
3,VTDAELAEVLAR,192000000.0,1.600000e+07
4,STEFDNLLLVGPLAGK,78300.0,4.893750e+03


### Exporting txt files of stripped peptides at confidence cutoffs:

In [8]:
# keep only peptide column peptides <1% FDR (this is what we exported)
pep322moddup = peaksdb322[["Peptide"]]

# keep only the stripped peptide column <1% FDR
# this is what we'll use for UniPept input, etc
pep322dup = peaksdb322[["stripped_peptide"]]

# deduplicate both of these lists
pep322mod = pep322moddup.drop_duplicates()
pep322 = pep322dup.drop_duplicates()

# print out the #s of modified and stripped peptides, deduplicated and not

print("Total modified PeaksDB peptides in 322:", len(pep322moddup))
print("Deduplicated modified PeaksDB peptides in 322:", len(pep322mod))
print("Total stripped PeaksDB peptides in 322:", len(pep322dup))
print("Deduplicated stripped PeaksDB peptides in 322:", len(pep322))

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing
pep322.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDB/TW_322_T0_trypsin_combine_PTMopt_DB_FDR1_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDB/TW_322_T0_trypsin_combine_PTMopt_DB_FDR1_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDB/TW_322_T0_trypsin_combine_PTMopt_DB_FDR1_stripped_peptides.fas

# look at the stripped peptides
pep322.head()

Total modified PeaksDB peptides in 322: 826
Deduplicated modified PeaksDB peptides in 322: 826
Total stripped PeaksDB peptides in 322: 826
Deduplicated stripped PeaksDB peptides in 322: 791


,stripped_peptide
0,LPQVEGTGGDVQPSQDLVR
1,VIGQNEAVDAVSNAIR
2,AIDLIDEAASSIR
3,VTDAEIAEVLAR
4,STEFDNILIVGPIAGK


## NAAF correction and exporting files with AA and PTM totals:

In [10]:
# made a new dataframe that contains the sums of certain columns in the modified
# peptide dataframe above 

index = ['sample total']

data = {'A': peaksdb322['A'].sum(),
        'C': peaksdb322['C'].sum(),
        'D': peaksdb322['D'].sum(),
        'E': peaksdb322['E'].sum(),
        'F': peaksdb322['F'].sum(),
        'G': peaksdb322['G'].sum(),
        'H': peaksdb322['H'].sum(),
        'I': peaksdb322['I'].sum(),
        'K': peaksdb322['K'].sum(),
        'L': peaksdb322['L'].sum(),
        'M': peaksdb322['M'].sum(),
        'N': peaksdb322['N'].sum(),
        'P': peaksdb322['P'].sum(),
        'Q': peaksdb322['Q'].sum(),
        'R': peaksdb322['R'].sum(),
        'S': peaksdb322['S'].sum(),
        'T': peaksdb322['T'].sum(),
        'V': peaksdb322['V'].sum(),
        'W': peaksdb322['W'].sum(),
        'Y': peaksdb322['Y'].sum(),
        'c-carb': peaksdb322['c-carb'].sum(),
        'm-oxid': peaksdb322['m-oxid'].sum(),
        'k-oxid': peaksdb322['k-oxid'].sum(),
        'p-oxid': peaksdb322['p-oxid'].sum(),
        'r-oxid': peaksdb322['r-oxid'].sum(),
        'y-oxid': peaksdb322['y-oxid'].sum(),
        'n-deam': peaksdb322['n-deam'].sum(),
        'k-meth': peaksdb322['k-meth'].sum(),
        'r-meth': peaksdb322['r-meth'].sum(),
        'q-pyro': peaksdb322['q-pyro'].sum(),
        'k-acet': peaksdb322['k-acet'].sum(),
        'Total area': peaksdb322['Area'].sum(),
        'Total length': peaksdb322['stripped_length'].sum()
       }

totalpeaksdb322 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                              'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                              'c-carb', 'm-oxid', 'k-oxid', 'p-oxid', 'r-oxid', \
                                              'y-oxid', 'n-deam', 'k-meth', 'r-meth', 'q-pyro', \
                                              'k-acet', 'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaksdb322['% C w/ carb'] = totalpeaksdb322['c-carb'] / totalpeaksdb322['C'] 

# calculate percentage of M's that are oxidized
totalpeaksdb322['% M w/ oxid'] = totalpeaksdb322['m-oxid'] / totalpeaksdb322['M'] 

# calculate percentage of K's that are oxidized
totalpeaksdb322['% K w/ oxid'] = totalpeaksdb322['k-oxid'] / totalpeaksdb322['K'] 

# calculate percentage of P's that are oxidized
totalpeaksdb322['% P w/ oxid'] = totalpeaksdb322['p-oxid'] / totalpeaksdb322['P'] 

# calculate percentage of R's that are oxidized
totalpeaksdb322['% R w/ oxid'] = totalpeaksdb322['p-oxid'] / totalpeaksdb322['R'] 

# calculate percentage of Y's that are oxidized
totalpeaksdb322['% Y w/ oxid'] = totalpeaksdb322['y-oxid'] / totalpeaksdb322['Y'] 

# calculate percentage of N's that are deamidated
totalpeaksdb322['% N w/ deam'] = totalpeaksdb322['n-deam'] / totalpeaksdb322['N'] 

# calculate percentage of K's that are methylated
totalpeaksdb322['% K w/ meth'] = totalpeaksdb322['k-meth'] / totalpeaksdb322['K'] 

# calculate percentage of R's that are methylated
totalpeaksdb322['% R w/ meth'] = totalpeaksdb322['r-meth'] / totalpeaksdb322['R'] 

# calculate percentage of Q's that are pyro glu'd
totalpeaksdb322['% Q w/ pyro'] = totalpeaksdb322['q-pyro'] / totalpeaksdb322['Q'] 

# calculate percentage of K's that are acetylation
totalpeaksdb322['% K w/ acet'] = totalpeaksdb322['k-acet'] / totalpeaksdb322['K'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaksdb322['NAAF denom.'] = totalpeaksdb322['Total area'] / totalpeaksdb322['Total length']

# write modified dataframe to new txt file
totalpeaksdb322.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDB/TW_322_T0_trypsin_combine_PTMopt_DB_FDR1_totals.csv")

totalpeaksdb322.head()

,A,C,D,E,F,G,H,I,K,L,...,% K w/ oxid,% P w/ oxid,% R w/ oxid,% Y w/ oxid,% N w/ deam,% K w/ meth,% R w/ meth,% Q w/ pyro,% K w/ acet,NAAF denom.
sample total,591,112,614,625,307,979,132,539,402,631,...,0.004975,0.03876,0.05305,0.020243,0.038647,0.0199,0.002653,0.020619,0.0,555224.331024


In [12]:
# use the calculated NAAF factor (in totalpeaksdb322 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

NAAF20 = 555224.331024

# we'll use -10lgP > 20 as an approximate cutoff for the <1.0% FDR which is what we actually have
peaksdb322['NAAF factor'] = (peaksdb322['NAAF_num.'])/NAAF20

# make a dataframe that contains only what we need: sequences, AAs, PTMs
peaksdb322_NAAF = peaksdb322[['stripped_peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', \
                              'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                              'c-carb', 'm-oxid', 'k-oxid', 'p-oxid', 'r-oxid', 'y-oxid', 'n-deam', \
                              'k-meth', 'r-meth', 'q-pyro', 'k-acet']].copy()

# multiply the NAAF20 factor by the AAs to normalize its abundance by peak area and peptide length

peaksdb322_NAAF['A-NAAF20'] = peaksdb322_NAAF['A'] * peaksdb322['NAAF factor']
peaksdb322_NAAF['C-NAAF20'] = peaksdb322_NAAF['C'] * peaksdb322['NAAF factor']
peaksdb322_NAAF['D-NAAF20'] = peaksdb322_NAAF['D'] * peaksdb322['NAAF factor']
peaksdb322_NAAF['E-NAAF20'] = peaksdb322_NAAF['E'] * peaksdb322['NAAF factor']
peaksdb322_NAAF['F-NAAF20'] = peaksdb322_NAAF['F'] * peaksdb322['NAAF factor']
peaksdb322_NAAF['G-NAAF20'] = peaksdb322_NAAF['G'] * peaksdb322['NAAF factor']
peaksdb322_NAAF['H-NAAF20'] = peaksdb322_NAAF['H'] * peaksdb322['NAAF factor']
peaksdb322_NAAF['I-NAAF20'] = peaksdb322_NAAF['I'] * peaksdb322['NAAF factor']
peaksdb322_NAAF['K-NAAF20'] = peaksdb322_NAAF['K'] * peaksdb322['NAAF factor']
peaksdb322_NAAF['L-NAAF20'] = peaksdb322_NAAF['L'] * peaksdb322['NAAF factor']
peaksdb322_NAAF['M-NAAF20'] = peaksdb322_NAAF['M'] * peaksdb322['NAAF factor']
peaksdb322_NAAF['N-NAAF20'] = peaksdb322_NAAF['N'] * peaksdb322['NAAF factor']
peaksdb322_NAAF['P-NAAF20'] = peaksdb322_NAAF['P'] * peaksdb322['NAAF factor']
peaksdb322_NAAF['Q-NAAF20'] = peaksdb322_NAAF['Q'] * peaksdb322['NAAF factor']
peaksdb322_NAAF['R-NAAF20'] = peaksdb322_NAAF['R'] * peaksdb322['NAAF factor']
peaksdb322_NAAF['S-NAAF20'] = peaksdb322_NAAF['S'] * peaksdb322['NAAF factor']
peaksdb322_NAAF['T-NAAF20'] = peaksdb322_NAAF['T'] * peaksdb322['NAAF factor']
peaksdb322_NAAF['V-NAAF20'] = peaksdb322_NAAF['V'] * peaksdb322['NAAF factor']
peaksdb322_NAAF['W-NAAF20'] = peaksdb322_NAAF['W'] * peaksdb322['NAAF factor']
peaksdb322_NAAF['Y-NAAF20'] = peaksdb322_NAAF['Y'] * peaksdb322['NAAF factor']

# multiply the NAAF20 factor by the PTMs normalize its abundance by peak area and peptide length

peaksdb322_NAAF['ccarb-NAAF20'] = peaksdb322_NAAF['c-carb'] * peaksdb322_NAAF['NAAF factor']
peaksdb322_NAAF['moxid-NAAF20'] = peaksdb322_NAAF['m-oxid'] * peaksdb322_NAAF['NAAF factor']
peaksdb322_NAAF['koxid-NAAF20'] = peaksdb322_NAAF['k-oxid'] * peaksdb322_NAAF['NAAF factor']
peaksdb322_NAAF['poxid-NAAF20'] = peaksdb322_NAAF['p-oxid'] * peaksdb322_NAAF['NAAF factor']
peaksdb322_NAAF['roxid-NAAF20'] = peaksdb322_NAAF['r-oxid'] * peaksdb322_NAAF['NAAF factor']
peaksdb322_NAAF['yoxid-NAAF20'] = peaksdb322_NAAF['y-oxid'] * peaksdb322_NAAF['NAAF factor']
peaksdb322_NAAF['ndeam-NAAF20'] = peaksdb322_NAAF['n-deam'] * peaksdb322_NAAF['NAAF factor']
peaksdb322_NAAF['kmeth-NAAF20'] = peaksdb322_NAAF['k-meth'] * peaksdb322_NAAF['NAAF factor']
peaksdb322_NAAF['rmeth-NAAF20'] = peaksdb322_NAAF['r-meth'] * peaksdb322_NAAF['NAAF factor']
peaksdb322_NAAF['qpyro-NAAF20'] = peaksdb322_NAAF['q-pyro'] * peaksdb322_NAAF['NAAF factor']
peaksdb322_NAAF['kacet-NAAF20'] = peaksdb322_NAAF['k-acet'] * peaksdb322_NAAF['NAAF factor']

# write the dataframe to a new csv
peaksdb322_NAAF.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDB/TW_322_T0_trypsin_combine_PTMopt_DB_FDR1_NAAF.csv")

peaksdb322_NAAF.head()

,stripped_peptide,NAAF factor,A,C,D,E,F,G,H,I,...,moxid-NAAF20,koxid-NAAF20,poxid-NAAF20,roxid-NAAF20,yoxid-NAAF20,ndeam-NAAF20,kmeth-NAAF20,rmeth-NAAF20,qpyro-NAAF20,kacet-NAAF20
0,LPQVEGTGGDVQPSQDLVR,12.702310,0,0,2,1,0,3,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,VIGQNEAVDAVSNAIR,15.984530,3,0,1,1,0,1,0,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,AIDLIDEAASSIR,23.691048,3,0,2,1,0,0,0,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,VTDAEIAEVLAR,28.817181,3,0,1,2,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,STEFDNILIVGPIAGK,0.008814,1,0,1,1,1,2,0,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# made a new dataframe that contains the sums of NAAF normalized AAs for peaksdb322 peaksdb results
# also contains the sums of the NAAF-corrected PTMs occurances for each affected residue

index = ['sample total']

data = {'NAAF': peaksdb322_NAAF['NAAF factor'].sum(),
        'A': peaksdb322_NAAF['A-NAAF20'].sum(),
        'C': peaksdb322_NAAF['C-NAAF20'].sum(),
        'D': peaksdb322_NAAF['D-NAAF20'].sum(),
        'E': peaksdb322_NAAF['E-NAAF20'].sum(),
        'F': peaksdb322_NAAF['F-NAAF20'].sum(),
        'G': peaksdb322_NAAF['G-NAAF20'].sum(),
        'H': peaksdb322_NAAF['H-NAAF20'].sum(),
        'I': peaksdb322_NAAF['I-NAAF20'].sum(),
        'K': peaksdb322_NAAF['K-NAAF20'].sum(),
        'L': peaksdb322_NAAF['L-NAAF20'].sum(),
        'M': peaksdb322_NAAF['M-NAAF20'].sum(),
        'N': peaksdb322_NAAF['N-NAAF20'].sum(),
        'P': peaksdb322_NAAF['P-NAAF20'].sum(),
        'Q': peaksdb322_NAAF['Q-NAAF20'].sum(),
        'R': peaksdb322_NAAF['R-NAAF20'].sum(),
        'S': peaksdb322_NAAF['S-NAAF20'].sum(),
        'T': peaksdb322_NAAF['T-NAAF20'].sum(),
        'V': peaksdb322_NAAF['V-NAAF20'].sum(),
        'W': peaksdb322_NAAF['W-NAAF20'].sum(),
        'Y': peaksdb322_NAAF['Y-NAAF20'].sum(),
        'c-carb': peaksdb322_NAAF['ccarb-NAAF20'].sum(),
        'm-oxid': peaksdb322_NAAF['moxid-NAAF20'].sum(),
        'k-oxid': peaksdb322_NAAF['koxid-NAAF20'].sum(),
        'p-oxid': peaksdb322_NAAF['poxid-NAAF20'].sum(),
        'r-oxid': peaksdb322_NAAF['roxid-NAAF20'].sum(),
        'y-oxid': peaksdb322_NAAF['yoxid-NAAF20'].sum(),
        'n-deam': peaksdb322_NAAF['ndeam-NAAF20'].sum(),
        'k-meth': peaksdb322_NAAF['kmeth-NAAF20'].sum(),
        'r-meth': peaksdb322_NAAF['rmeth-NAAF20'].sum(),
        'q-pyro': peaksdb322_NAAF['qpyro-NAAF20'].sum(),
        'k-acet': peaksdb322_NAAF['kacet-NAAF20'].sum()
       }

totalpeaksdb322_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', \
                                                   'G', 'H', 'I','K', 'L', 'M', \
                                                   'N', 'P', 'Q', 'R', 'S', \
                                                   'T', 'V', 'W', 'Y', 'c-carb', \
                                                   'm-oxid', 'k-oxid', 'p-oxid', 'r-oxid', \
                                                   'y-oxid', 'n-deam', 'k-meth', 'r-meth', \
                                                   'q-pyro', \
                                                   'k-acet'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q, K, and Rs


# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaksdb322_NAAF['% C w/ carb.'] = totalpeaksdb322_NAAF['c-carb'] / totalpeaksdb322_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaksdb322_NAAF['% M w/ oxid'] = totalpeaksdb322_NAAF['m-oxid'] / totalpeaksdb322_NAAF['M'] 

# calculate NAAF-corrected percentage of K's that are oxidized
totalpeaksdb322_NAAF['% K w/ oxid'] = totalpeaksdb322_NAAF['k-oxid'] / totalpeaksdb322_NAAF['K'] 

# calculate NAAF-corrected percentage of P's that are oxidized
totalpeaksdb322_NAAF['% P w/ oxid'] = totalpeaksdb322_NAAF['p-oxid'] / totalpeaksdb322_NAAF['P'] 

# calculate NAAF-corrected percentage of R's that are oxidized
totalpeaksdb322_NAAF['% R w/ oxid'] = totalpeaksdb322_NAAF['r-oxid'] / totalpeaksdb322_NAAF['R'] 

# calculate NAAF-corrected percentage of Y's that are oxidized
totalpeaksdb322_NAAF['% Y w/ oxid'] = totalpeaksdb322_NAAF['y-oxid'] / totalpeaksdb322_NAAF['Y'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb322_NAAF['% N w/ deam'] = totalpeaksdb322_NAAF['n-deam'] / totalpeaksdb322_NAAF['N'] 

# calculate NAAF-corrected percentage of K's that are methylated
totalpeaksdb322_NAAF['% K w/ meth'] = totalpeaksdb322_NAAF['k-meth'] / totalpeaksdb322_NAAF['K'] 

# calculate NAAF-corrected percentage of R's that are methylated
totalpeaksdb322_NAAF['% R w/ meth'] = totalpeaksdb322_NAAF['r-meth'] / totalpeaksdb322_NAAF['R'] 

# calculate NAAF-corrected percentage of Q's that are pyro glu'd
totalpeaksdb322_NAAF['% Q w/ pyro'] = totalpeaksdb322_NAAF['q-pyro'] / totalpeaksdb322_NAAF['Q'] 

# calculate NAAF-corrected percentage of K's that are methylated
totalpeaksdb322_NAAF['% K w/ acet'] = totalpeaksdb322_NAAF['k-acet'] / totalpeaksdb322_NAAF['K'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaksdb322_NAAF['NAAF check'] = totalpeaksdb322_NAAF['NAAF'] / 555224.331024

# write modified dataframe to new txt file, same name + totals
totalpeaksdb322_NAAF.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDB/TW_322_T0_trypsin_combine_PTMopt_DB_FDR1_NAAF_totals.csv")

totalpeaksdb322_NAAF.head()

,NAAF,A,C,D,E,F,G,H,I,K,...,% K w/ oxid,% P w/ oxid,% R w/ oxid,% Y w/ oxid,% N w/ deam,% K w/ meth,% R w/ meth,% Q w/ pyro,% K w/ acet,NAAF check
sample total,851.851461,733.850015,22.962069,477.439217,666.681224,183.171804,693.180906,151.1454,620.656959,170.375682,...,0.000089,0.005893,0.000648,0.001996,0.001846,0.005538,0.000174,0.002328,0.0,0.001534
